# Plotly advanced examples

This notebook demonstrates advanced Plotly features: subplots with secondary y-axis, range sliders/selectors, faceting, animations, 3D surfaces, and buttons for interactive trace toggling.

Per request, the first cell installs plotly.

In [1]:
pip install -q plotly

Note: you may need to restart the kernel to use updated packages.


Import libraries and prepare the output folder where we will save HTML plots and CSV data so results are reproducible outside the notebook as well.

In [2]:
import os
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

os.makedirs("outputs", exist_ok=True)
np.random.seed(0)

Create a synthetic time series dataset with multiple categories and save it to CSV for later reuse and inspection of intermediate results.

In [3]:
dates = pd.date_range("2021-01-01", periods=90, freq="D")
categories = ["A", "B", "C"]
frames = []
for i, c in enumerate(categories):
    t = np.linspace(0, 4*np.pi, len(dates))
    value = np.sin(t + 0.5*i) + 0.1*np.random.randn(len(dates)) + 0.2*i
    measure2 = np.cos(0.5*t + 0.3*i) + 0.1*np.random.randn(len(dates))
    frames.append(pd.DataFrame({"date": dates, "category": c, "value": value, "measure2": measure2}))
df = pd.concat(frames, ignore_index=True)

df.to_csv("outputs/advanced_timeseries.csv", index=False)

Build subplots with a secondary y-axis: one line trace plus a 7-day rolling mean as bars. Save the interactive result to HTML (no need for extra engines).

In [4]:
dfa = df[df["category"] == "A"].copy()
dfa["rolling7"] = dfa["value"].rolling(7, min_periods=1).mean()

fig_sub = make_subplots(specs=[[{"secondary_y": True}]])
fig_sub.add_trace(
    go.Scatter(x=dfa["date"], y=dfa["value"], name="A: value", mode="lines"),
    secondary_y=False,
)
fig_sub.add_trace(
    go.Bar(x=dfa["date"], y=dfa["rolling7"], name="7d rolling"),
    secondary_y=True,
)
fig_sub.update_yaxes(title_text="value", secondary_y=False)
fig_sub.update_yaxes(title_text="rolling mean", secondary_y=True)
fig_sub.update_layout(title="Category A: value vs rolling mean", template="plotly_white", height=420)
fig_sub.write_html("outputs/advanced_subplots.html", include_plotlyjs="cdn")

Enhance the subplot figure with a range slider and range selector buttons, and save again as a separate HTML file to compare versions step-by-step.

In [5]:
fig_sub.update_xaxes(rangeslider_visible=True)
fig_sub.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=[
                dict(count=7, label="1w", step="day", stepmode="backward"),
                dict(count=14, label="2w", step="day", stepmode="backward"),
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(step="all"),
            ]
        )
    )
)
fig_sub.write_html("outputs/advanced_subplots_controls.html", include_plotlyjs="cdn")

Create a faceted scatter plot with marginal distributions, using a custom continuous color scale. Save to HTML for interactive inspection of hover info and facets.

In [6]:
fig_facet = px.scatter(
    df,
    x="value",
    y="measure2",
    color="date",
    facet_col="category",
    opacity=0.75,
    color_continuous_scale="Viridis",
    marginal_x="histogram",
    marginal_y="violin",
    render_mode="webgl",
    title="Facet scatter with marginals and continuous color",
)
fig_facet.update_layout(template="plotly_white", height=450)
fig_facet.write_html("outputs/advanced_facet_scatter.html", include_plotlyjs="cdn")

Animate a scatter plot over time (frame per day) to visualize category evolution. Save as HTML so the play/pause controls work in-browser without the notebook runtime.

In [7]:
xrange = [float(df["value"].min()) - 0.2, float(df["value"].max()) + 0.2]
yrange = [float(df["measure2"].min()) - 0.2, float(df["measure2"].max()) + 0.2]
fig_anim = px.scatter(
    df,
    x="value",
    y="measure2",
    animation_frame="date",
    animation_group="category",
    color="category",
    range_x=xrange,
    range_y=yrange,
    title="Animated scatter over time by category",
)
fig_anim.update_layout(template="plotly_white", height=450)
fig_anim.write_html("outputs/advanced_animation.html", include_plotlyjs="cdn")

Create a 3D surface plot from a synthetic function z = sin(x) * cos(y). This demonstrates 3D interactions and custom color scales. Save to HTML for rotation and zoom in the browser.

In [8]:
x = np.linspace(-2*np.pi, 2*np.pi, 60)
y = np.linspace(-2*np.pi, 2*np.pi, 60)
X, Y = np.meshgrid(x, y)
Z = np.sin(X) * np.cos(Y)
fig_surf = go.Figure(go.Surface(z=Z, x=X, y=Y, colorscale="Turbo"))
fig_surf.update_layout(
    title="3D surface: sin(x) * cos(y)",
    scene=dict(xaxis_title="x", yaxis_title="y", zaxis_title="z"),
    template="plotly_white",
    height=500,
)
fig_surf.write_html("outputs/advanced_surface.html", include_plotlyjs="cdn")

Use Graph Objects with update menus (buttons) to toggle which category traces are visible. This is useful for dashboards where users need to filter traces interactively. Save to HTML to test the buttons in the browser.

In [9]:
fig_btn = go.Figure()
for c in categories:
    d = df[df["category"] == c]
    fig_btn.add_trace(go.Scatter(x=d["date"], y=d["value"], mode="lines", name=c))

buttons = []
buttons.append(dict(label="All", method="update", args=[{"visible": [True]*len(categories)}, {"title": "All categories"}]))
for i, c in enumerate(categories):
    vis = [False]*len(categories)
    vis[i] = True
    buttons.append(dict(label=c, method="update", args=[{"visible": vis}, {"title": f"Category {c}"}]))

fig_btn.update_layout(
    title="Toggle category visibility via buttons",
    template="plotly_white",
    height=420,
    updatemenus=[dict(type="buttons", buttons=buttons, x=1.05, y=1, xanchor="left")],
)
fig_btn.write_html("outputs/advanced_buttons.html", include_plotlyjs="cdn")